# Obtención de datos meteorológicos con la API de la AEMET

Constará de varias funciones para obtener los datos históricos a partir del 22 de junio de la API (en la zona de Cangas de Onís como municipio), y luego las previsiones para los siguientes días (en la zona de montaña de los Picos de Europa). Se construirán algunos dataframe con los que podremos construir un CSV y exportarlo. 

In [15]:
# Paquetes
from api_key import api_key
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

## Definición de constantes

In [16]:
querystring = {"api_key": api_key}
headers = {
	'cache-control': "no-cache"
}

ID_MUNICIPIO = '33012'  # Cangas de Onís
ZONA = 'peu1'  # Zona montañosa Picos de Europa
IDEMA = "1178R"  # Sotres

fecha_inicio = "2025-06-22T00:00:00UTC"
fecha_fin = "2025-06-30T00:00:00UTC"

## Obtención de valores climatológicos pasados

Esta función usa el endpoint de la AEMET para obtener los valores climatológicos por días observados en una determinada estación, dentro de un rango de días especificado. Devuelve el dataframe con las columnas de interés.

In [17]:
def consulta_historicos(inicio, fin, estacion=IDEMA):
	url = f"https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/{inicio}/fechafin/{fin}/estacion/{estacion}"

	response = requests.request("GET", url, headers=headers, params=querystring)

	if response.status_code == 200:

		# Descarga de los datos reales
		datos = response.json()
		url_datos = datos["datos"]
		response_datos = requests.request("GET", url_datos, headers=headers)

		if response_datos.status_code == 200:
			prediccion = response_datos.json()

			# Convertir a dataframe y poner la fecha en el índice
			df = pd.DataFrame(prediccion)
			if "fecha" in df.columns:
				df["fecha"] = pd.to_datetime(df["fecha"])

			# Eliminar columnas que no nos interesan
			df_nuevo = df.drop(columns=["indicativo", "nombre", "provincia", "altitud", "horatmin", "horatmax", "tmed", "horaracha", "horaPresMax", "horaPresMin", "horaHrMax", "horaHrMin"])
			
			return df_nuevo

In [ ]:
pd.set_option('display.max_columns', None)      # Muestra todas las columnas
pd.set_option('display.width', None)            # Usa el ancho completo del terminal o Jupyter
pd.set_option('display.expand_frame_repr', False)  # Evita que pandas divida líneas

df_pasado = consulta_historicos(fecha_inicio, fecha_fin)
print(df_pasado)

       fecha  prec  tmin  tmax dir velmedia racha presMax presMin hrMedia hrMax hrMin
0 2025-06-22   0,0  11,9  20,4  15      1,4   7,2   884,2   881,9      99   100    61
1 2025-06-23   2,0  11,7  25,0  24      2,5  16,1   884,0   882,2      64   100    40
2 2025-06-24  23,8  13,7  26,6  23      5,6  21,7   884,7   878,7      54    94    34
3 2025-06-25   2,0  11,5  19,9  12      3,6  14,7   882,5   875,5      86   100    50
4 2025-06-26   0,0  11,4  18,2  07      1,4   6,1   886,9   882,5      95   100    80
5 2025-06-27   0,0  14,8  23,5  06      2,5   7,5   889,2   886,2      58    88    51


## Obtención de previsiones en zona de montaña